In [1]:
#Modules to install via pip pandas,ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pprint import pprint
import os
import import_ipynb
import sys
import kmeans
sys.path.append('../')
from functions import *
from trace_analysis import *
from plots import *
from trace_analysis_cooja2 import *
from node import *
from plots_analysis import *
from pandas.plotting import scatter_matrix
import cmath as math
from mpl_toolkits.mplot3d import Axes3D
from sklearn import cluster
from sklearn.metrics import confusion_matrix
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.cluster import KMeans
# scipy
from scipy.cluster.vq import kmeans,vq,whiten
import sklearn.metrics as sm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
%matplotlib inline
import random
random.seed(6666)

In [2]:
directory="../cooja3-9nodes/"
plots = [
        #2 BH3
        (directory+"traces/1bh-3", 'grid9_1bh-3_2019-02-13_16:28_',"BH"),
        (directory+"traces/1bh-3", 'grid9_1bh-3_2019-02-13_22:05_',"BH"),
        #2 BH5
         (directory+"traces/1bh-5", 'grid9_1bh-5_2019-02-13_15:31_',"BH"),
          (directory+"traces/1bh-5", 'grid9_1bh-5_2019-02-13_21:44_',"BH"),
        #2 BH 6
        (directory+"traces/1bh-6", 'grid9_1bh-6_2019-02-13_12:59_',"BH"),
        (directory+"traces/1bh-6", 'grid9_1bh-6_2019-02-13_19:15_',"BH"),
        #2 BH 7
         (directory+"traces/1bh-7", 'grid9_1bh-7_2019-02-13_15:08_',"BH"),
         (directory+"traces/1bh-7", 'grid9_1bh-7_2019-02-13_20:02_',"BH"),
         #2 bh 9
         (directory+"traces/1bh-9", 'grid9_1bh-9_2019-02-13_15:57_',"BH"),
         (directory+"traces/1bh-9", 'grid9_1bh-9_2019-02-13_19:35_',"BH"),
         #3 normal
         (directory+"traces/normal", 'grid9_normal_2019-02-13_17:05_',"normal"),
         (directory+"traces/normal",  "grid9_normal_2019-02-13_18:51_","normal"),
         (directory+"traces/normal",  "grid9_normal_2019-02-13_22:23_","normal"),
        ]

print(directory)
cases=[]
casesAccuracy=[]
for row in plots:
    cases.append(row[1])
    casesAccuracy.append(row[2])




data=import_Cooja2(plots)



../cooja3-9nodes/


In [3]:
for i in range(len(data)):
    for j in range(len(data[i])):
        if (i==12): print(data[i][j].pkts)

     seq    rtt
0      1  110.0
1      3  225.0
2      5  105.0
3      6   89.6
4      7   80.3
5      8   71.2
6      9   66.4
7     10   77.9
8     11   81.8
9     12   83.7
10    13   91.8
11    14   94.1
12    15   89.8
13    16   75.1
14    17   83.8
15    18  136.0
16    19   64.8
17    20   60.9
18    21   65.7
19    22   78.3
20    23   69.4
21    24   67.8
22    25   82.1
23    26   58.9
24    27  143.0
25    28  156.0
26    29   86.7
27    30  169.0
28    31  132.0
29    32  265.0
..   ...    ...
165  170   94.3
166  171  126.0
167  172  167.0
168  173   70.0
169  174  145.0
170  175  142.0
171  176   69.5
172  177  152.0
173  178   81.7
174  179  126.0
175  180   76.9
176  181  148.0
177  182  137.0
178  183  121.0
179  184  257.0
180  185  203.0
181  186  169.0
182  187  144.0
183  188  135.0
184  189  145.0
185  190  160.0
186  191  159.0
187  192   84.6
188  193  213.0
189  194  195.0
190  195  143.0
191  196  154.0
192  197  161.0
193  198   73.5
194  199   97.2

[195 ro

In [19]:
pings=getPings(data)
#All data collection is in variable node that is a list of list of nodes
#3 nets input x 9 nodes by net

d={ "label":[],
   "type":[],
    "count":[],
    "std":  [],
    "mean": [],
    "var":  [],
    #"25%":  [],
    #"50%":[],
    #"75%":[],
   #"min":[],
   #"max":[],
   "hop":[],
   #"missing":[],
   "packet loss":[],
   "outliers":[],
   "node":[]
}
#count=[]
labels=[]
var=[]
#window=100
#stats=pd.DataFrame(columns=columns)

for i in range(len(data)):
    #window=pings[i]
    
    for j in range(len(data[i])):
        #n=pings[i]
        n=100
        window=50
        #print(n)
        for z in range(0,n,int(window)):
            #if(z+window>n):break
            #print(z,z+window)
            
            #df1 = df1.assign(e=p.Series(np.random.randn(sLength)).values)
            node=data[i][j].pkts
            name=str(j)+" "+cases[i]
            nodeWindow=node[(node["seq"]<z+window) & (node["seq"]>=z)]
            
            nodeWindowP=nodeWindow["rtt"]
            #df1 = df1.assign(e=p.Series(np.random.randn(sLength)).values)
            #if(nodeWindowP.count()<window/2): break
            #current=data[i][j].pkts['rtt'][z:z+window]
            #print(nodeWindow.shape)
            d["count"].append(nodeWindowP.count())
            #Case without outliers
            #d["std"].append(getStdValues(data[i][j].pkts)["rtt"].std())

            #Case with outliers
            std=0
            if (nodeWindowP.std()>10):
                std=1
                std=nodeWindowP.std()
                
            d["std"].append(std)
            mean=nodeWindowP.mean()
            #if(mean<1):print(mean)
            d["mean"].append(mean)
            var=1
            if (nodeWindowP.var()>var): var=nodeWindowP.var()
            d["var"].append(var)
            d["label"].append(cases[i])
            #d["25%"].append(nodeWindow.describe()["25%"])
            #d["50%"].append(nodeWindow.describe()["50%"])
            #d["75%"].append(nodeWindow.describe()["75%"])
            #d["min"].append(nodeWindow.describe()["min"])
            #d["max"].append(nodeWindow.describe()["max"])
            d["hop"].append(data[i][j].hop)
            d["type"].append(casesAccuracy[i])
            d["outliers"].append(getOutliers(nodeWindow)["rtt"].count())
            missing=window-nodeWindow.count()
            d["node"].append(data[i][j].ip)
            mP=getPercentageMissingPackets(nodeWindow,window)
            PL=0
            if(mP>30):
                PL=1
                PL=mP
            d["packet loss"].append(PL)

            
            
stats=pd.DataFrame(d)            


        

#correction=np.array(correction)
#correction_alt=np.array(correction_alt)

dataK=stats.drop([
    "label",
    "mean",
    "var",
    "std",
    #"packet loss",
    "outliers",
    "hop",
    "count",
    "node",
    #"type"
],axis=1)
  
print(dataK.shape)
#dataK=dataK.dropna()
dataK=dataK.fillna(0)
print(dataK.shape)
print(dataK.head())      

(234, 2)
(234, 2)
  type  packet loss
0   BH         98.0
1   BH        100.0
2   BH         98.0
3   BH        100.0
4   BH         98.0


In [6]:
stats

,label,type,count,std,mean,var,hop,packet loss,outliers,node,predicted,predicted number,correction number
0,grid9_1bh-3_2019-02-13_16:28_,BH,1,0.000000,1.000000,1.000000,4,98.0,0,aaaa::212:7408:8:808,normal,1,0
1,grid9_1bh-3_2019-02-13_16:28_,BH,0,0.000000,NaN,1.000000,4,100.0,0,aaaa::212:7408:8:808,normal,1,0
2,grid9_1bh-3_2019-02-13_16:28_,BH,1,0.000000,1.000000,1.000000,3,98.0,0,aaaa::212:7407:7:707,normal,1,0
3,grid9_1bh-3_2019-02-13_16:28_,BH,0,0.000000,NaN,1.000000,3,100.0,0,aaaa::212:7407:7:707,normal,1,0
4,grid9_1bh-3_2019-02-13_16:28_,BH,1,0.000000,1.000000,1.000000,2,98.0,0,aaaa::212:7402:2:202,normal,1,0
5,grid9_1bh-3_2019-02-13_16:28_,BH,0,0.000000,NaN,1.000000,2,100.0,0,aaaa::212:7402:2:202,normal,1,0
6,grid9_1bh-3_2019-02-13_16:28_,BH,1,0.000000,1.000000,1.000000,2,98.0,0,aaaa::212:7406:6:606,normal,1,0
7,grid9_1bh-3_2019-02-13_16:28_,BH,0,0.000000,NaN,1.000000,2,100.0,0,aaaa::212:7406:6:606,normal,1,0
8,grid9_1bh-3_2019-02-13_16:28_,BH,1,0.000000,1.000000,1.000000,3,98.0,0,aaaa::212:7409:9:909,normal,1,0
9,grid9_1bh-3_2019-02-13_16:28_,BH,0,0.000000,NaN,1.000000,3,100.0,0,aaaa::212:7409:9:909,normal,1,0


In [20]:




correction=[]
correction_alt=[] #fr 3 cluster 0:normal net 1:bh net 2:bh
col=np.array(dataK["type"])
dataK=dataK.drop(["type"],axis=1)
#Creating simple array to correct unsupervised learning
#NB as it is unsupervised could happen that the correction are inverted
for i in range(len(col)):
    el=d["type"][i]
    if el=="normal":
        correction.append(1)
        correction_alt.append(0)
        
    else:
        #print(el=="BH2" and i==BlackHole[2])
        correction.append(0)
        correction_alt.append(1)
                        

dataC=stats["label"]

#Y = data[['var']]

#X = data[['std']]

kmeans = KMeans(n_clusters=2)
kmeans.fit(dataK)
labels = kmeans.predict(dataK)
centroids = kmeans.cluster_centers_
labels=accuracy_score_corrected(correction,labels)
predicted=[]
for i in range(len(labels)):
    
    if(labels[i]==1):
        predicted.append("normal")
    else: predicted.append("BH")

#print(len(predicted))
stats["predicted"]=pd.Series(np.array(predicted))
stats["predicted number"]=pd.Series(np.array(labels))
stats["correction number"]=pd.Series(np.array(correction))
stats_csv=stats[[
    "label",
    "type",
    "predicted",
    "packet loss",
    "outliers",
    "std",
    "hop",
    "node",
    "mean"
      
      
      ]]
stats_csv.to_csv("results_kmeans.csv", sep='\t', encoding='utf-8')
stats.head()


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]
0.47863247863247865
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]
array([[ 58, 122],
       [ 

,label,type,count,std,mean,var,hop,packet loss,outliers,node,predicted,predicted number,correction number
0,grid9_1bh-3_2019-02-13_16:28_,BH,1,0.0,1.0,1.0,4,98.0,0,aaaa::212:7408:8:808,BH,0,0
1,grid9_1bh-3_2019-02-13_16:28_,BH,0,0.0,NaN,1.0,4,100.0,0,aaaa::212:7408:8:808,BH,0,0
2,grid9_1bh-3_2019-02-13_16:28_,BH,1,0.0,1.0,1.0,3,98.0,0,aaaa::212:7407:7:707,BH,0,0
3,grid9_1bh-3_2019-02-13_16:28_,BH,0,0.0,NaN,1.0,3,100.0,0,aaaa::212:7407:7:707,BH,0,0
4,grid9_1bh-3_2019-02-13_16:28_,BH,1,0.0,1.0,1.0,2,98.0,0,aaaa::212:7402:2:202,BH,0,0


In [21]:
net_results={
   "case":[],
    "predicted":[],
    "real":[]
}
#print(stats["predicted number"])
for case in range(len(cases)):
    subset=stats[stats["label"]==cases[case]]
    mean_predicted=str(subset["predicted number"].mean()*100)+"% normal"
    net_results["case"].append(cases[case])
    net_results["predicted"].append(mean_predicted)
    net_results["real"].append(casesAccuracy[case])

    
    
results=pd.DataFrame(net_results)  

results

,case,predicted,real
0,grid9_1bh-3_2019-02-13_16:28_,11.11111111111111% normal,BH
1,grid9_1bh-3_2019-02-13_22:05_,11.11111111111111% normal,BH
2,grid9_1bh-5_2019-02-13_15:31_,94.44444444444444% normal,BH
3,grid9_1bh-5_2019-02-13_21:44_,83.33333333333334% normal,BH
4,grid9_1bh-6_2019-02-13_12:59_,88.88888888888889% normal,BH
5,grid9_1bh-6_2019-02-13_19:15_,61.111111111111114% normal,BH
6,grid9_1bh-7_2019-02-13_15:08_,88.88888888888889% normal,BH
7,grid9_1bh-7_2019-02-13_20:02_,72.22222222222221% normal,BH
8,grid9_1bh-9_2019-02-13_15:57_,88.88888888888889% normal,BH
9,grid9_1bh-9_2019-02-13_19:35_,77.77777777777779% normal,BH


In [8]:
X = dataK.iloc[:,0:4]
pca = PCA(n_components=2)
pca.fit(X)
X_ = pca.transform(X)

dfPCA = pd.DataFrame({'x1': X_[:,0], 'x2': X_[:,1]})
dfPCA['labels'] = stats['label']
dfPCA.head()

labels = stats['label'].unique().tolist()
plt.figure(figsize=(15,8))
for lab in labels:
    plt.scatter(dfPCA.loc[dfPCA['labels'] == lab, 'x1'],  dfPCA.loc[dfPCA['labels'] == lab, 'x2'], label=lab, alpha=0.7)
    plt.legend()


ValueError: n_components=2 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [ ]:
print(centroids[0]
     )

In [ ]:
x= stats["packet loss"]
y=stats["var"]
plt.figure(figsize=(15,8))
labels = stats['label'].unique().tolist()
plt.figure(figsize=(15,8))
for lab in labels:
    plt.scatter(stats.loc[stats['label'] == lab, 'packet loss'],  stats.loc[stats['label'] == lab, 'packet loss'], label=lab, alpha=0.7)
    plt.legend()

# Different types of clustering


In [15]:
def accuracy_score_corrected(correction,labels):
    print(np.array(correction))
    labels_alt=[]
    sum_labels=0
    sum_labels_alt=0
    for el in labels:
        if (el==0):
            labels_alt.append(1)
            sum_labels_alt+=1
        elif el==1:
            labels_alt.append(0)
            sum_labels+=1

    accuracy=sm.accuracy_score(correction, labels)
    accuracy_alt=sm.accuracy_score(correction, labels_alt)
    #print(correction)
    
    
    if (sum_labels>sum_labels_alt):
        print(accuracy)

    else:
        print(accuracy_alt)
        labels=labels_alt
    print(np.array(labels))
    confusionMatrix=sm.confusion_matrix(correction, labels)

    pprint(confusionMatrix)
    return labels


In [16]:
stats=pd.DataFrame(d)  
dataK=stats.drop([
    "label",
    "mean",
    "var",
    "std",
    #"packet loss",
    "outliers",
    "hop",
    "count",
    "node",
    #"type"
],axis=1)
  
print(dataK.shape)
#dataK=dataK.dropna()
dataK=dataK.fillna(0)
print(dataK.shape)
print(dataK.head())  

correction=[]
correction_alt=[] #fr 3 cluster 0:normal net 1:bh net 2:bh
col=np.array(dataK["type"])
dataK=dataK.drop(["type"],axis=1)
#Creating simple array to correct unsupervised learning
#NB as it is unsupervised could happen that the correction are inverted
for i in range(len(col)):
    el=d["type"][i]
    if el=="normal":
        correction.append(1)
        correction_alt.append(0)
        
    else:
        #print(el=="BH2" and i==BlackHole[2])
        correction.append(0)
        correction_alt.append(1)
                        

dataC=stats["label"]

#Y = data[['var']]

#X = data[['std']]






(234, 2)
(234, 2)
  type  packet loss
0   BH         98.0
1   BH        100.0
2   BH         98.0
3   BH        100.0
4   BH         98.0


In [17]:
std = StandardScaler()
dataK_scaled=std.fit_transform(dataK)
#print(dataK_scaled)
dataK_scaled = pd.DataFrame(dataK_scaled, columns = dataK.columns)
print(dataK_scaled.head())
kmeans = KMeans(n_clusters = 2, init = 'k-means++', n_init = 10, max_iter = 300, tol = 1e-04, random_state = 0)
kmeans.fit_predict(dataK)
labels=kmeans.labels_
labels=accuracy_score_corrected(correction,labels)
#accuracy=sm.accuracy_score(correction, labels)
#confusionMatrix=sm.confusion_matrix(correction, labels)
#pprint(accuracy)
#print(correction)
#print(labels)
centroids = kmeans.cluster_centers_
predicted=[]
for i in range(len(labels)):
    
    if(labels[i]==1):
        predicted.append("normal")
    else: predicted.append("BH")

        
stats["predicted"]=pd.Series(np.array(predicted))
stats["predicted number"]=pd.Series(np.array(labels))
stats["correction number"]=pd.Series(np.array(correction))
stats_csv=stats[[
    "label",
    "type",
    "predicted",
    "packet loss",
    "outliers",
    "std",
    "hop",
    "node",
    "mean"
      
      
      ]]
print(stats_csv.head())
stats_csv.to_csv("results_kmeans.csv", sep='\t', encoding='utf-8')

   packet loss
0     1.828643
1     1.879217
2     1.828643
3     1.879217
4     1.828643
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]
0.47863247863247865
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [18]:
net_results={
   "case":[],
    "predicted":[],
    "real":[]
}
#print(stats["predicted number"])
for case in range(len(cases)):
    subset=stats[stats["label"]==cases[case]]
    mean_predicted=str(subset["predicted number"].mean()*100)+"% normal"
    net_results["case"].append(cases[case])
    net_results["predicted"].append(mean_predicted)
    net_results["real"].append(casesAccuracy[case])

    
    
results=pd.DataFrame(net_results)  

results

,case,predicted,real
0,grid9_1bh-3_2019-02-13_16:28_,11.11111111111111% normal,BH
1,grid9_1bh-3_2019-02-13_22:05_,11.11111111111111% normal,BH
2,grid9_1bh-5_2019-02-13_15:31_,94.44444444444444% normal,BH
3,grid9_1bh-5_2019-02-13_21:44_,83.33333333333334% normal,BH
4,grid9_1bh-6_2019-02-13_12:59_,88.88888888888889% normal,BH
5,grid9_1bh-6_2019-02-13_19:15_,61.111111111111114% normal,BH
6,grid9_1bh-7_2019-02-13_15:08_,88.88888888888889% normal,BH
7,grid9_1bh-7_2019-02-13_20:02_,72.22222222222221% normal,BH
8,grid9_1bh-9_2019-02-13_15:57_,88.88888888888889% normal,BH
9,grid9_1bh-9_2019-02-13_19:35_,77.77777777777779% normal,BH
